In [70]:
##Machine LEarning LAnguage Translation

In [71]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense

batch_size = 64 #batch size for training
epochs = 100 # no of epocs to train for
latent_dim =256 #latent dimensionality of the encoding space
num_samples = 10000 # no of samples to train on
data_path = 'C:/Users/akirt/hin-eng/hin.txt' #dataset path

In [72]:
#vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path,'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples,len(lines)-1)]:
    input_text,target_text,_ = line.split('\t')
    #We use 'tab' as the 'start sequence' character
    #for the targets, and '\n' as 'end sequence' character
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char) 

In [73]:
input_characters

{' ',
 '!',
 '"',
 '$',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '€'}

In [74]:
input_texts

['Wow!',
 'Duck!',
 'Duck!',
 'Help!',
 'Jump.',
 'Jump.',
 'Jump.',
 'Hello!',
 'Hello!',
 'Cheers!',
 'Cheers!',
 'Exhale.',
 'Exhale.',
 'Got it?',
 "I'm OK.",
 'Inhale.',
 'Inhale.',
 'Thanks!',
 'We won.',
 'Awesome!',
 'Come in.',
 'Get out!',
 'Go away!',
 'Goodbye!',
 'Perfect!',
 'Perfect!',
 'We lost.',
 'Welcome.',
 'Welcome.',
 'Have fun.',
 'Have fun.',
 'Have fun.',
 'I forgot.',
 'I forgot.',
 "I'll pay.",
 "I'm fine.",
 "I'm full.",
 "Let's go!",
 'Pick Tom.',
 'Answer me.',
 'Birds fly.',
 'Excuse me.',
 'Fantastic!',
 'I fainted.',
 'I fear so.',
 'I laughed.',
 "I'm alone.",
 "I'm alone.",
 "I'm bored.",
 "I'm broke.",
 "I'm tired.",
 "It's cold.",
 'Well done!',
 'Who knows?',
 'Who knows?',
 'Who knows?',
 'Who knows?',
 'Wonderful!',
 'Birds sing.',
 'Come on in.',
 'Definitely!',
 "Don't move.",
 'Fire burns.',
 'Follow him.',
 'I can swim.',
 'I can swim.',
 'I love you.',
 'I love you.',
 'I love you.',
 'I love you.',
 'I love you.',
 "I'm coming.",
 "I'm hung

In [75]:
target_texts

['\tवाह!\n',
 '\tझुको!\n',
 '\tबतख़!\n',
 '\tबचाओ!\n',
 '\tउछलो.\n',
 '\tकूदो.\n',
 '\tछलांग.\n',
 '\tनमस्ते।\n',
 '\tनमस्कार।\n',
 '\tवाह-वाह!\n',
 '\tचियर्स!\n',
 '\tसांस छोड़।\n',
 '\tसांस छोड़ो।\n',
 '\tसमझे कि नहीं?\n',
 '\tमैं ठीक हूँ।\n',
 '\tसांस ले।\n',
 '\tसांस लो।\n',
 '\tधन्यवाद!\n',
 '\tहम जीते।\n',
 '\tबहुत बढ़िया!\n',
 '\tअंदर आ जाओ।\n',
 '\tबाहर निकल जाओ!\n',
 '\tचले जाओ!\n',
 '\tख़ुदा हाफ़िज़।\n',
 '\tउत्तम!\n',
 '\tसही!\n',
 '\tहम हार गए।\n',
 '\tआपका स्वागत है।\n',
 '\tस्वागतम्।\n',
 '\tमज़े करना।\n',
 '\tमौज करना।\n',
 '\tमज़े करो।\n',
 '\tमैं भूल गया।\n',
 '\tमैं भूल गई।\n',
 '\tमैं पैसे दूंगा।\n',
 '\tमैं ठीक हूँ।\n',
 '\tमेरा पेट भर गया है।\n',
 '\tचलो चलें!\n',
 '\tटॉम उठाओ।\n',
 '\tमुझे जवाब दो।\n',
 '\tपंछी उड़ते हैं।\n',
 '\tमाफ़ कीजिए।\n',
 '\tबहुत ख़ूब!\n',
 '\tमैं बेहोश हो गया।\n',
 '\tखेद की बात है, लेकिन वैसा ही है।\n',
 '\tमैं हँसा।\n',
 '\tमैं अकेला हूँ ।\n',
 '\tमैं अकेली हूँ ।\n',
 '\tमैं बोर हो रहा हूँ।\n',
 '\tमेरा दीवालिया हो चुका है।\n',
 '\tमैं 

In [76]:
len(target_characters)

93

In [77]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [78]:
print('Number of samples:',len(input_texts)) #number of sentences
print('Number of unique input tokens:',num_encoder_tokens)
print('Number of unique output tokens:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_length) #english here
print('Max Sequence length for outputs:',max_decoder_seq_length) #hindi here

Number of samples: 3116
Number of unique input tokens: 70
Number of unique output tokens: 93
Max Sequence length for inputs: 107
Max Sequence length for outputs: 123


In [79]:
#input_characters → a list of unique characters in your dataset (e.g., ["a","b","c","d",...]).
#enumerate(input_characters) → gives pairs (index, char).
#The comprehension flips it into (char, index) → so each character gets a unique integer.
#dict() → stores that mapping as a dictionary.

input_token_index =dict(
    [(char,i) for i,char in enumerate(input_characters)]
)
target_token_index =dict(
    [(char,i) for i,char in enumerate(target_characters)]
)

In [80]:
input_token_index #how many english characters are present based on soretd order index will be given

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 "'": 4,
 ',': 5,
 '-': 6,
 '.': 7,
 '0': 8,
 '1': 9,
 '2': 10,
 '3': 11,
 '4': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'W': 41,
 'Y': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68,
 '€': 69}

In [81]:
target_token_index #how many hindi characters are present based on soretd order index will be given

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '7': 12,
 '9': 13,
 ':': 14,
 '?': 15,
 'A': 16,
 'B': 17,
 'I': 18,
 '|': 19,
 'ँ': 20,
 'ं': 21,
 'ः': 22,
 'अ': 23,
 'आ': 24,
 'इ': 25,
 'ई': 26,
 'उ': 27,
 'ऊ': 28,
 'ऋ': 29,
 'ए': 30,
 'ऐ': 31,
 'ऑ': 32,
 'ओ': 33,
 'औ': 34,
 'क': 35,
 'ख': 36,
 'ग': 37,
 'घ': 38,
 'च': 39,
 'छ': 40,
 'ज': 41,
 'झ': 42,
 'ञ': 43,
 'ट': 44,
 'ठ': 45,
 'ड': 46,
 'ढ': 47,
 'ण': 48,
 'त': 49,
 'थ': 50,
 'द': 51,
 'ध': 52,
 'न': 53,
 'प': 54,
 'फ': 55,
 'ब': 56,
 'भ': 57,
 'म': 58,
 'य': 59,
 'र': 60,
 'ल': 61,
 'व': 62,
 'श': 63,
 'ष': 64,
 'स': 65,
 'ह': 66,
 '़': 67,
 'ा': 68,
 'ि': 69,
 'ी': 70,
 'ु': 71,
 'ू': 72,
 'ृ': 73,
 'ॅ': 74,
 'े': 75,
 'ै': 76,
 'ॉ': 77,
 'ो': 78,
 'ौ': 79,
 '्': 80,
 '।': 81,
 '०': 82,
 '१': 83,
 '२': 84,
 '४': 85,
 '५': 86,
 '६': 87,
 '७': 88,
 '८': 89,
 '९': 90,
 '\u200d': 91,
 '€': 92}

In [82]:
#encoder_input_data all zeroes values to our vector with 3 d dimensions of total number of sentencesi.e. no of pairs
#,max english sentence length, number of english characters num_samples = len(input_texts)
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length, num_encoder_tokens),
    dtype='float32'
)
#decoder_input_data is a 3D array of shape (num_pairs, max_hindi_sentence_length, num_hindi_characters) 
#containg a one-hot vectorization of the hindi sentences.
decoder_input_data = np.zeros(
    (len(input_texts),max_decoder_seq_length, num_decoder_tokens),
    dtype='float32'
)
#decoder_target_data is the same as decoder_input_data but offset by one timestep.
#decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].
decoder_target_data = np.zeros(
    (len(input_texts),max_decoder_seq_length, num_decoder_tokens),
    dtype='float32'
)

In [83]:
input_texts

['Wow!',
 'Duck!',
 'Duck!',
 'Help!',
 'Jump.',
 'Jump.',
 'Jump.',
 'Hello!',
 'Hello!',
 'Cheers!',
 'Cheers!',
 'Exhale.',
 'Exhale.',
 'Got it?',
 "I'm OK.",
 'Inhale.',
 'Inhale.',
 'Thanks!',
 'We won.',
 'Awesome!',
 'Come in.',
 'Get out!',
 'Go away!',
 'Goodbye!',
 'Perfect!',
 'Perfect!',
 'We lost.',
 'Welcome.',
 'Welcome.',
 'Have fun.',
 'Have fun.',
 'Have fun.',
 'I forgot.',
 'I forgot.',
 "I'll pay.",
 "I'm fine.",
 "I'm full.",
 "Let's go!",
 'Pick Tom.',
 'Answer me.',
 'Birds fly.',
 'Excuse me.',
 'Fantastic!',
 'I fainted.',
 'I fear so.',
 'I laughed.',
 "I'm alone.",
 "I'm alone.",
 "I'm bored.",
 "I'm broke.",
 "I'm tired.",
 "It's cold.",
 'Well done!',
 'Who knows?',
 'Who knows?',
 'Who knows?',
 'Who knows?',
 'Wonderful!',
 'Birds sing.',
 'Come on in.',
 'Definitely!',
 "Don't move.",
 'Fire burns.',
 'Follow him.',
 'I can swim.',
 'I can swim.',
 'I love you.',
 'I love you.',
 'I love you.',
 'I love you.',
 'I love you.',
 "I'm coming.",
 "I'm hung

In [84]:
#one hot reperesentation
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    
    # --- Encoder input ---
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    # Pad remaining encoder timesteps with space character
    for t in range(len(input_text), max_encoder_seq_length):
        encoder_input_data[i, t, input_token_index[' ']] = 1.0

    # --- Decoder input & target ---
    for t, char in enumerate(target_text):
        # decoder input  # decoder_input_data gets the character at timestep t
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        
        # decoder target (shifted by 1 timestep)
        if t > 0:
            decoder_target_data[i, t-1, target_token_index[char]] = 1.0

    # Pad remaining decoder timesteps with space character
    for t in range(len(target_text), max_decoder_seq_length):
        decoder_input_data[i, t, target_token_index[' ']] = 1.0
        decoder_target_data[i, t, target_token_index[' ']] = 1.0

In [85]:
encoder_input_data[0].shape

(107, 70)

In [86]:
#define an input sequence and process it
#None → variable sequence length (timesteps)
#num_encoder_tokens → number of features per timestep (one-hot vector size)
#LSTM(latent_dim, return_state=True):
#latent_dim → number of LSTM units (hidden size)
#return_state=True → returns the last hidden state (state_h) and cell state (state_c) along with outputs
#encoder_outputs → output at all timesteps (we discard it for basic seq2seq)
#state_h, state_c → final hidden and cell states → used to initialize decoder
#We save the states as a list, which will be passed to the decoder to provide initial context.

encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder = LSTM(latent_dim,return_state=True)
encoder_outputs, state_h,state_c = encoder(encoder_inputs)
#we discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h,state_c]

In [87]:
#set up the decoder, using 'encoder_states' as initial state
#Placeholder for decoder input sequences (one-hot encoded).
#Shape: (timesteps, features) → (None, num_decoder_tokens)
#During training, this is usually the target sequence shifted by one timestep (teacher forcing).
decoder_inputs = Input(shape=(None,num_decoder_tokens))
#we set up our decoder to return full output sentences, and to return internal states as well. 
#We dont use the return states in the traaining model, but we will use them in inference
#return_sequences=True → we want output at every timestep (for one-hot prediction of each character).
#return_state=True → returns (outputs, state_h, state_c)
#We ignore the states during training because we only need the full outputs.
#During inference, these states will be critical to feed the LSTM step by step.
#initial_state=encoder_states → initializes decoder LSTM with encoder context, so decoder knows about input sequence
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
#Dense(num_decoder_tokens) → output at each timestep is a probability distribution over all possible target characters.
#softmax → gives probabilities for one-hot classification.
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)
#Teacher forcing: decoder input = ground truth shifted, decoder target = ground truth.
#States ignored in training → only needed during inference for step-by-step prediction.

In [88]:
# encoder_input_data = np.array(encoder_input_data, dtype="float32")
# decoder_input_data = np.array(decoder_input_data, dtype="float32")
# decoder_target_data = np.array(decoder_target_data, dtype="float32")
print("encoder_input_data.shape =", encoder_input_data.shape)
print("decoder_input_data.shape =", decoder_input_data.shape)
print("decoder_target_data.shape =", decoder_target_data.shape)

encoder_input_data.shape = (3116, 107, 70)
decoder_input_data.shape = (3116, 123, 93)
decoder_target_data.shape = (3116, 123, 93)


In [89]:
#define the model that will turn encoder_input_data & decoder_input_data into
#decoder_target_data
model = Model([encoder_inputs,decoder_inputs],decoder_outputs)
#Given an input sentence (encoder_inputs) 
#and a shifted version of the output sentence (decoder_inputs), predict the actual target sentence (decoder_target_data).

In [90]:
#Here's the steps:
#1. encode input and retrieve initial decoder state
#2. run one step of decoder with this initial state and a 'start of sequence' token as target
#output will be the next target token
#3. repeat with the current target token and current states

#define sampling models / inference models

#Encoder model for inference
#Takes the input sentence (already preprocessed as one-hot/embedding).
#Returns the final hidden and cell states of the encoder LSTM (state_h, state_c).
#These states are the context vector → a compressed representation of the input sentence.
encoder_model = Model(encoder_inputs, encoder_states)

#Decoder model for inference
#These are placeholders for decoder’s previous states (hidden + cell state).
#At inference time, we feed them back into the decoder at each timestep.
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
#The decoder takes:the current input token (e.g., <start> or the last predicted word),the previous states.
#Produces: a probability distribution over the next token (decoder_outputs),new hidden and cell states (state_h, state_c).
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

#This defines the step-by-step decoder used in inference.You call this repeatedly in a loop (like in your decode_sequence function).
#Each time, you feed the predicted token back as input for the next step.
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

#reverse-lookup token index to decode sequences back to something readable
#These are index → character mappings (the reverse of input_token_index and target_token_index).
#After decoding, you can convert a sequence of predicted indices back into human-readable text.
reverse_input_char_index =dict(
    (i,char) for char,i in input_token_index.items())
reverse_target_char_index =dict(
    (i,char) for char,i in target_token_index.items())


In [91]:
from tensorflow.keras.callbacks import Callback
#After every epoch, print some fixed test inputs
class TranslatorMonitor(Callback):
    def __init__(self, test_inputs, decode_fn):
        super().__init__()
        self.test_inputs = test_inputs
        self.decode_fn = decode_fn

    def on_epoch_end(self, epoch, logs=None):
        print("\n--- Epoch %d ---" % (epoch + 1))
        for inp in self.test_inputs:
            # prepare encoder input
            input_seq = encoder_input_data[input_texts.index(inp): input_texts.index(inp)+1]
            decoded_sentence = self.decode_fn(input_seq)
            print(f"Input: {inp} -> Decoded: {decoded_sentence}")

Epoch 1/100


C:\Users\akirt\AppData\Roaming\Python\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_27', 'keras_tensor_31']. Received: the structure of inputs=('*', '*')
  warnings.warn(


39/39 ━━━━━━━━━━━━━━━━━━━━ 76s 2s/step - accuracy: 0.7137 - loss: 1.8536 - val_accuracy: 0.6762 - val_loss: 1.9464
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.8008 - loss: 1.0255 - val_accuracy: 0.6779 - val_loss: 1.7171
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.8019 - loss: 0.8758 - val_accuracy: 0.6786 - val_loss: 1.5231
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 0.8027 - loss: 0.8261 - val_accuracy: 0.6786 - val_loss: 1.4329
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.7989 - loss: 0.8164 - val_accuracy: 0.6631 - val_loss: 1.3511
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.8013 - loss: 0.8114 - val_accuracy: 0.6786 - val_loss: 1.3309
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 0.8019 - loss: 0.7825 - val_accuracy: 0.6787 - val_loss: 1.3162
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.8023 - loss: 0.7782 - val_accuracy: 0.6789 - val_loss: 1.3

In [ ]:
monitor = TranslatorMonitor(test_inputs, decode_sequence)

In [ ]:
#run training
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,
         batch_size=batch_size,
         epochs=epochs,
         validation_split=0.2,
         callbacks=[monitor])

In [ ]:
#This will let us see side-by-side input vs predicted translation for the first 100 samples.
for seq_index in range(100):
    #take one sequence (part of training set) for trying out decoding
    input_seq = encoder_input_data[seq_index : seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence: ',input_texts[seq_index])
    print('Decoded sentence: ',decoded_sentence)

In [92]:
#this is the inference-time decoding loop (also called sampling loop) for our sequence-to-sequence model
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    #Decoder always starts with a special start token ('\t'). Example: target sequence starts as just '\t'.
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    #decoding loop: Feed the previous token + previous states into the decoder. Decoder predicts probabilities over all possible next characters.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character. 
        #Stop when either:Model predicts the special end token ('\n'), OR Decoded sentence is longer than max length.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1). Convert predicted index → actual character (using reverse_target_char_index).
        #Append that character to your decoded string.
        #Example: Model predicts index 1 → reverse_target_char_index[1] = 'A'.decoded_sentence = "A".
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        #The predicted character becomes the next input token.The hidden states get updated for the next loop.
        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
#This will let us see side-by-side input vs predicted translation for the first 100 samples.
for seq_index in range(100):
    #take one sequence (part of training set) for trying out decoding
    input_seq = encoder_input_data[seq_index : seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence: ',input_texts[seq_index])
    print('Decoded sentence: ',decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step


C:\Users\akirt\AppData\Roaming\Python\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_31', 'keras_tensor_36', 'keras_tensor_37']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
-
Input sentence:  Wow!
Decoded sentence:  मैं उसके पास करते हों।

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━

In [ ]:
#Training phase
#What happens?
#The model sees many input–output pairs (your dataset).
#It adjusts its weights using backpropagation and an optimizer (like RMSprop/Adam).
#Example in your seq2seq model:
#Encoder input = "Hello"
#Decoder input = "\tBonjour" (start token + partial output) Decoder target = "Bonjour\n" (true full output shifted by one step)
#Loss (categorical_crossentropy) is computed between predicted output and target.Model learns patterns (e.g., language translation rules).

In [ ]:
#3Inference phase What happens?
#After training, you want to generate outputs for new, unseen inputs.
#The decoder does not know the full target sequence anymore (we don’t have ground truth at test time).
#Instead: Feed an input sentence to the encoder → get hidden states.
#Start the decoder with the special start token (\t).
#The decoder predicts the next character/word.
#That predicted token is fed back into the decoder as the next input.Repeat until an end token (\n) or max length is reached.
#This is step-by-step generation instead of parallel training.

In [ ]:
# Let's make 3 pairs of simple sentences (English -> Hindi transliteration style)
#input_texts = ["hi", "how", "bye"]
#target_texts = ["\tनमस्ते\n", "\tकैसे\n", "\tअलविदा\n"]


In [ ]:
### Step 0: Make a tiny toy dataset (English -> Hindi-like tokens) input_texts: ['hi', 'how', 'bye'] target_texts: ['\tनमस्ते\n', '\tकैसे\n', '\tअलविदा\n'] 
### Step 1: Build character vocabularies input_characters: ['b', 'e', 'h', 'i', 'o', 'w', 'y'] target_characters: ['\t', '\n', 'अ', 'क', 'त', 'द', 'न', 'म', 'ल', 'व', 'स', 'ा', 'ि', 'े', 'ै', '्'] num_encoder_tokens: 7 num_decoder_tokens: 16 max_encoder_seq_length: 3 max_decoder_seq_length: 8 
### Step 2: Create char->index and index->char maps input_token_index (sample): [('b', 0), ('e', 1), ('h', 2), ('i', 3), ('o', 4)] target_token_index (sample): [('\t', 0), ('\n', 1), ('अ', 2), ('क', 3), ('त', 4)] 
### Step 3: Allocate one-hot arrays encoder_input_data.shape = (3, 3, 7) decoder_input_data.shape = (3, 8, 16) decoder_target_data.shape = (3, 8, 16) 
### Step 4: Vectorize (fill one-hot tensors) One-hot example (encoder_input_data[0]): [[0. 0. 1. 0. 0. 0. 0.] [0. 0. 0. 1. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0.]] One-hot example (decoder_input_data[0]): [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] One-hot example (decoder_target_data[0]): [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 
### Step 5: Try to build & train a tiny seq2seq model

In [ ]:
'''
Unique input chars: ['b', 'e', 'h', 'i', 'l', 'o', 'y']
Unique target chars: ['\t', '\n', 'त', 'न', 'ब', 'म', 'य', 'ल', 'स', 'ह', 'ा', 'े', 'ो', '्']
Encoder tokens: 7
Decoder tokens: 14
Max input length: 5
Max target length: 8

Shapes:
encoder_input_data: (3, 5, 7)
decoder_input_data: (3, 8, 14)
decoder_target_data: (3, 8, 14)
2025-08-26 16:24:20.447097: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Model: "functional"
┌─────────────────────┬───────────────────┬────────────┬───────────────────┐
│ Layer (type)        │ Output Shape      │    Param # │ Connected to      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, None, 7)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 14)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 64),      │     18,432 │ input_layer[0][0] │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │     20,224 │ input_layer_1[0]… │
│                     │ 64), (None, 64),  │            │ lstm[0][1],       │
│                     │ (None, 64)]       │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 14)  │        910 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘
 🔹 1. Dataset
input_texts  = ["hi", "how", "bye"]
target_texts = ["\tनमस्ते\n", "\tकैसे\n", "\tअलविदा\n"]


We add \t as the start token, and \n as the end token in each target.

🔹 2. Vocabulary
Encoder side (English):
input_characters = ['b','e','h','i','o','w','y']
len = 7

Decoder side (Hindi-like chars):

From targets → ['\t','\n','न','म','स','्','त','े','क','ै','अ','ल','व','ि','द','ा','स']
(16–17 unique chars, let’s say 17 total)

So:

num_encoder_tokens = 7
num_decoder_tokens = 17

🔹 3. Sequence lengths

Max encoder length = 3 (how)

Max decoder length = 8 (\tअलविदा\n)

🔹 4. Vectorization shapes
encoder_input_data: (3, 3, 7)   # (samples, max_eng_len, eng_vocab)
decoder_input_data: (3, 8, 17)  # (samples, max_hin_len, hin_vocab)
decoder_target_data: (3, 8, 17) # shifted by 1

🔹 5. Example One-hot Encoding

Example "hi":

encoder_input_data[0] =
[[0,0,1,0,0,0,0],   # 'h'
 [0,0,0,1,0,0,0],   # 'i'
 [0,0,0,0,0,0,0]]   # pad


Example target " \tनमस्ते\n":
decoder_input_data[0] (input side of teacher forcing)

t=0: "\t"
t=1: "न"
t=2: "म"
t=3: "स"
t=4: "्"
t=5: "त"
t=6: "े"
t=7: "\n"


👉 decoder_target_data[0] is the same but shifted:

t=0 target: "न"
t=1 target: "म"
...
t=6 target: "\n"

🔹 6. Model Shapes

Encoder input shape: (None, 3, 7)

Encoder LSTM (latent_dim=64):

Outputs: (None, 64) (final state_h, state_c)

Decoder input shape: (None, 8, 17)

Decoder LSTM output: (None, 8, 64)

Dense: (None, 8, 17) → softmax over 17 chars

🔹 7. Training (Teacher Forcing)

For "hi" → "\tनमस्ते\n":

Step 0: input="\t", target="न"
Step 1: input="न", target="म"
Step 2: input="म", target="स"
Step 3: input="स", target="्"
Step 4: input="्", target="त"
Step 5: input="त", target="े"
Step 6: input="े", target="\n"


Loss = crossentropy(predicted softmax, true one-hot)

🔹 8. Inference (Greedy Decoding)

Suppose we give "hi".

Step 1: Encode "hi"
encoder_input = (1, 3, 7)
encoder_model → (state_h, state_c), each (1, 64)

Step 2: Start decoder with "\t"
target_seq = (1, 1, 17) one-hot for "\t"

Step 3: Predict next char
decoder_model.predict([target_seq, state_h, state_c])
→ output_tokens shape (1, 1, 17)
→ e.g. softmax = [0.1, 0.0, 0.6, 0.1, ...]
→ argmax = "न"


Decoded sentence so far: "न"

Step 4: Feed back "न"
target_seq = one-hot("न")


Predict next char → "म"

Step 5: Continue
"स" → "्" → "त" → "े" → "\n"


Output: "नमस्ते"

🔹 Example Full Inference Outputs

Input: "hi" → "नमस्ते"

Input: "how" → "कैसे"

Input: "bye" → "अलविदा"

✅ Would you like me to actually show numeric one-hot → softmax → argmax for one full decoding step (say "hi" first step " \t" → "न") with dummy probabilities, so you see real numbers flowing?

You said:
yes
ChatGPT said:

Perfect 👍 let’s do a numerical dry run of inference for the input "hi" with your toy dataset.
We’ll go step by step, showing one-hot → LSTM → Dense → softmax → argmax.

🔹 Step 1: Encode "hi"

Vocabulary (English): ['b','e','h','i','o','w','y']

So "hi" → one-hot:

encoder_input = (1, 3, 7)
[
  [0,0,1,0,0,0,0],   # h
  [0,0,0,1,0,0,0],   # i
  [0,0,0,0,0,0,0]    # padding
]


👉 Encoder LSTM compresses this into final hidden states (state_h, state_c), each (1, 64) (numbers like [0.12, -0.08, …]).

We don’t need exact LSTM math here, just that "hi" → some vector representation.

🔹 Step 2: Start Decoder with "\t"

Hindi vocab size = 17.

One-hot of start token "\t":

target_seq = (1, 1, 17)
[
  [1,0,0,0,0,...,0]
]

🔹 Step 3: First Prediction (want "न")

Decoder LSTM takes:

inputs: target_seq("\t"), state_h, state_c
outputs: (decoder_output, new_state_h, new_state_c)


decoder_output → shape (1, 1, 64)

Dense layer → (1, 1, 17)

Say Dense gives raw logits:

[ -1.2,  0.5,  2.3, -0.7,  0.1,  ... ]   # length 17


Softmax → probabilities:

[0.02, 0.05, 0.65, 0.04, 0.03, 0.02, 0.01, ...]


👉 argmax = index 2 = "न"

So predicted char = "न"

Decoded so far: "न"

🔹 Step 4: Feed back "न"

One-hot "न":

[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]


Pass again with updated states.

Logits →

[0.1, 0.2, 0.1, 1.9, -0.3, ...]


Softmax →

[0.05, 0.1, 0.05, 0.70, 0.05, ...]


👉 argmax = "म"

Decoded so far: "नम"

🔹 Step 5: Continue

Feed "म" → predict "स"

Feed "स" → predict "्"

Feed "्" → predict "त"

Feed "त" → predict "े"

Feed "े" → predict "\n" → STOP

Final decoded sequence: "नमस्ते"

✅ So the mechanics are:

One-hot input (\t)

LSTM + previous states

Dense → logits → softmax

Argmax = predicted char

Feed predicted char back

Repeat until "\n"

👉 Do you want me to also show the training loop dry run (teacher forcing), where instead of feeding the model’s own prediction, we feed the true Hindi char at each timestep and compute cross-entropy loss?

You said:
yes
ChatGPT said:

Great 👍 Let’s dry-run training with teacher forcing on your toy data:

input_texts  = ["hi", "how", "bye"]
target_texts = ["\tनमस्ते\n", "\tकैसे\n", "\tअलविदा\n"]

🔹 Step 1. Prepare Data Shapes

num_samples = 3

English vocab (input): ['b','e','h','i','o','w','y'] → num_encoder_tokens = 7

Hindi vocab (output): 17 unique chars → num_decoder_tokens = 17

max_encoder_seq_length = 3 (longest English = "how")

max_decoder_seq_length = 8 (longest Hindi = "अलविदा" + \t + \n)

So we have:

encoder_input_data : (3, 3, 7)
decoder_input_data : (3, 8, 17)
decoder_target_data: (3, 8, 17)

🔹 Step 2. Example Training Pair ("hi" → "नमस्ते")
Encoder Input

"hi" → one-hot shape (1, 3, 7):

[
  [0,0,1,0,0,0,0],   # h
  [0,0,0,1,0,0,0],   # i
  [0,0,0,0,0,0,0]    # padding
]

Decoder Input (with start token \t)

Target Hindi: "नमस्ते\n"

Add start/end → "\tनमस्ते\n"

One-hot each step:

Step 0: "\t" 
Step 1: "न"
Step 2: "म"
Step 3: "स"
Step 4: "्"
Step 5: "त"
Step 6: "े"
Step 7: "\n"


So decoder_input_data[0] = one-hot sequence of those 8 chars.

Decoder Target (shifted by +1)

Target is same sequence but shifted one step ahead:

Input (given to model)     → Target (expected next char)
---------------------------------------------------------
"\t"                       → "न"
"न"                        → "म"
"म"                        → "स"
"स"                        → "्"
"्"                        → "त"
"त"                        → "े"
"े"                        → "\n"
"\n"                       → padding


So decoder_target_data[0] = one-hot shifted version.

🔹 Step 3. Training Pass

At timestep 0:

Input to decoder: "\t"

Hidden state: from encoder("hi")

Dense softmax → distribution over 17 chars

Loss = crossentropy(predicted distribution, one-hot "न")

Backprop updates weights.

At timestep 1:

Input: "न"

Target: "म"

Model predicts distribution, compare with "म", compute loss.

Repeat until EOS "\n".

🔹 Step 4. Loss Computation (Toy Example)

Suppose at timestep 0, model outputs probabilities:

Predicted: [0.1, 0.05, 0.6, 0.1, ...]   # length 17
True:      [0,0,1,0,0,0,...]            # "न"


Cross-entropy loss:

Loss = -log(0.6) ≈ 0.51


If model had predicted 0.9 for "न", loss would be -log(0.9)=0.10 (much better).

The total loss = average of all timesteps.

🔹 Step 5. Epoch Training

For each epoch:

Take all (input_text, target_text) pairs

Run forward pass with teacher forcing (feed true previous token)

Compute total loss

Backprop through encoder + decoder

Update weights

After many epochs, model learns alignment:
"hi" → "नमस्ते", "how" → "कैसे", "bye" → "अलविदा"

✅ Key difference vs inference:

Training: Decoder sees the true token at each step. (Teacher forcing)

Inference: Decoder sees its own predicted token at each step.

Would you like me to numerically simulate one mini-training step (forward + loss calc + gradient intuition) with "hi" → "नमस्ते" so you can see how weights actually move?

You said:
yes
ChatGPT said:

Perfect 🙌 Let’s dry-run one mini training step on

English: "hi"
Hindi:   "नमस्ते"

🔹 Step 1. Shapes & Encodings

Encoder input: "hi" → (1, 3, 7)

Decoder input: "\tनमस्ते\n" → (1, 8, 17)

Decoder target: shifted sequence "नमस्ते\n" → (1, 8, 17)

🔹 Step 2. Forward Pass
Encoder

Feed "hi" character embeddings (7-dim one-hot) into encoder LSTM.

After 2 steps (h, i), final hidden state = context vector.
Example (dummy values):

Encoder hidden state h_enc = [0.3, -0.2, 0.5, ...]  (dim = 256)
Encoder cell state c_enc   = [0.1,  0.6, -0.1, ...]


This summarizes "hi".

Decoder Step 0 (input = "\t")

Input embedding = one-hot for \t.

Decoder LSTM takes: (embedding("\t"), h_enc, c_enc).

Outputs hidden state h₀ and softmax over 17 Hindi chars.

Suppose predicted distribution:

Predicted y₀ = [0.05, 0.1, 0.6, 0.05, ..., 0.01]  # length=17


True target = "न" → one-hot = [0,0,1,0,...].

Step 3. Loss Calculation

At t=0 (predict "न"):

Loss₀ = -log(pred["न"]) = -log(0.6) = 0.51


At t=1 (input "न", target "म")
Suppose model predicts "म" with prob 0.4:

Loss₁ = -log(0.4) = 0.92


At t=2 (input "म", target "स")
Suppose prob = 0.2:

Loss₂ = -log(0.2) = 1.61


Continue until EOS "\n".
Let’s say total losses per step = [0.51, 0.92, 1.61, 0.7, 0.3, 0.4, 0.2].

Final sequence loss:

L = (Σ Loss_t) / 7 = (4.64) / 7 ≈ 0.66

🔹 Step 4. Backpropagation

Cross-entropy gradients flow back:

From softmax → Dense layer → LSTM → encoder.

Example intuition:

At step 0, true label = "न", predicted prob = 0.6.
Gradient says: “Increase weight connecting hidden state → ‘न’ output; decrease weights for wrong chars.”

At step 2, predicted "स" with prob 0.2 (bad). Gradient pushes strongly to increase "स"’s weight next time.

Over epochs:

Encoder learns to encode "hi" → context vector that predicts start of "नमस्ते".

Decoder learns correct next-char probabilities.

🔹 Step 5. After Training

If trained well:

At inference, "hi" → encoder state.

Decoder starts with \t and predicts:

"न" (prob ~0.95)

feed back "न", predict "म" (prob ~0.9)

and so on, until EOS \n.

✅ That’s one training step dry-run:

Forward pass → predicted probabilities

Cross-entropy loss per timestep

Average loss

Backprop updates weights

Do you want me to numerically simulate how a weight value changes (e.g., one weight connecting hidden state → "न") after this step, so you see the update rule (w_new = w_old - lr * grad)?: 1.8818 - val_accuracy: 0.0000e+00 - val_loss: 1.3331
Let’s actually track one weight in this seq2seq model during training.
We’ll simplify a lot so it’s human-readable but still accurate.

🎯 Setup

We pick:

One weight: w = connection from decoder hidden unit h₁ → output character "न".

Learning rate: η = 0.1.

Decoder hidden activation at t=0 (when input = "\t") = h₁ = 0.5.

The output probability for "न" comes from softmax:

𝑝
(
"
न
"
)
=
𝑒
𝑧
न
∑
𝑗
𝑒
𝑧
𝑗
p("न")=
∑
j
	​

e
z
j
	​

e
z
न
	​

	​


where

𝑧
न
=
𝑤
⋅
ℎ
1
+
𝑏
z
न
	​

=w⋅h
1
	​

+b
🔹 Step 1. Forward Pass

Suppose:

Old weight = w_old = 0.2

Bias = 0

Hidden state = h₁ = 0.5

Then:

𝑧
न
=
0.2
⋅
0.5
=
0.1
z
न
	​

=0.2⋅0.5=0.1

Suppose denominator (all logits exponentiated and summed) = Σ e^{z_j} = 10.
So probability for "न":

𝑝
(
"
न
"
)
=
𝑒
0.1
10
≈
1.105
10
=
0.1105
p("न")=
10
e
0.1
	​

≈
10
1.105
	​

=0.1105
🔹 Step 2. Loss

True target at t=0 = "न" (so one-hot target = 1 for "न").
Cross-entropy loss:

𝐿
=
−
log
⁡
(
𝑝
(
"
न
"
)
)
=
−
log
⁡
(
0.1105
)
≈
2.20
L=−log(p("न"))=−log(0.1105)≈2.20
🔹 Step 3. Gradient

Gradient of loss wrt logit:

∂
𝐿
∂
𝑧
न
=
𝑝
(
"
न
"
)
−
1
=
0.1105
−
1
=
−
0.8895
∂z
न
	​

∂L
	​

=p("न")−1=0.1105−1=−0.8895

Gradient wrt weight:

∂
𝐿
∂
𝑤
=
(
𝑝
(
"
न
"
)
−
1
)
⋅
ℎ
1
=
−
0.8895
⋅
0.5
=
−
0.4448
∂w
∂L
	​

=(p("न")−1)⋅h
1
	​

=−0.8895⋅0.5=−0.4448
🔹 Step 4. Weight Update

Update rule:

𝑤
new
=
𝑤
old
−
𝜂
⋅
∂
𝐿
∂
𝑤
w
new
	​

=w
old
	​

−η⋅
∂w
∂L
	​

𝑤
new
=
0.2
−
0.1
⋅
(
−
0.4448
)
=
0.2
+
0.0445
=
0.2445
w
new
	​

=0.2−0.1⋅(−0.4448)=0.2+0.0445=0.2445
✅ Result

Before training: w = 0.200

After one gradient step: w = 0.245

Intuition: Because the model underpredicted "न" (p=0.11, true=1), the gradient pushed w upward, making "न" more likely next time.

Would you like me to continue this numeric dry-run across the next timestep (input "न", target "म") so you see how different weights get updated step by step?

You said:
yes
ChatGPT said:
Thought for a few seconds

Great — let’s continue the numeric dry-run for the next timestep (t = 1) where the decoder input is the model’s previous correct token "न" and the true target is "म". We'll follow the same simplified math as before so you can see how weights update timestep by timestep.

We'll track a single hidden unit activation h₁ and one weight per output character:

Previously we tracked wₙ (weight to output "न"). Now we’ll track wₘ (weight to output "म").

Learning rate η = 0.1.

Assumptions for t = 1 (made explicit)

Hidden activation at t=1: h₁ = 0.6 (could differ from t=0).

Current weight for "म" before this step: wₘ_old = 0.10.

Bias for that output = 0 (for simplicity).

For softmax denominator we assume Σ_j e^{z_j} = 8 (a simplifying assumption to get a concrete probability).
(these are toy numbers to illustrate gradient mechanics — real values come from the network's forward pass)

1) Compute logit for "म"
𝑧
म
=
𝑤
म
_
𝑜
𝑙
𝑑
⋅
ℎ
1
=
0.10
×
0.6
=
0.06
z
म
	​

=w
म_old
	​

⋅h
1
	​

=0.10×0.6=0.06
2) Softmax probability for "म"
𝑝
(
"म"
)
=
𝑒
𝑧
म
∑
𝑗
𝑒
𝑧
𝑗
≈
𝑒
0.06
8
p("म")=
∑
j
	​

e
z
j
	​

e
z
म
	​

	​

≈
8
e
0.06
	​


Numerically:

𝑒
0.06
≈
1.0618
⇒
𝑝
(
"म"
)
≈
1.0618
8
=
0.1327
e
0.06
≈1.0618⇒p("म")≈
8
1.0618
	​

=0.1327

(So the model currently gives ~13.3% probability to "म" at this timestep.)

3) Cross-entropy loss for this timestep

True target is "म" → one-hot for "म".

𝐿
1
=
−
log
⁡
𝑝
(
"म"
)
=
−
log
⁡
(
0.1327
)
≈
2.018
L
1
	​

=−logp("म")=−log(0.1327)≈2.018
4) Gradient of loss w.r.t. the logit 
𝑧
म
z
म
	​


For softmax + cross-entropy, gradient w.r.t. logit of the true class is:

∂
𝐿
∂
𝑧
म
=
𝑝
(
"म"
)
−
1
=
0.1327
−
1
=
−
0.8673
∂z
म
	​

∂L
	​

=p("म")−1=0.1327−1=−0.8673
5) Gradient w.r.t. weight 
𝑤
म
w
म
	​

∂
𝐿
∂
𝑤
म
=
∂
𝐿
∂
𝑧
म
⋅
∂
𝑧
म
∂
𝑤
म
=
(
𝑝
(
"म"
)
−
1
)
⋅
ℎ
1
=
−
0.8673
×
0.6
=
−
0.5204
∂w
म
	​

∂L
	​

=
∂z
म
	​

∂L
	​

⋅
∂w
म
	​

∂z
म
	​

	​

=(p("म")−1)⋅h
1
	​

=−0.8673×0.6=−0.5204
6) Update the weight (gradient descent)
𝑤
म
_
𝑛
𝑒
𝑤
=
𝑤
म
_
𝑜
𝑙
𝑑
−
𝜂
⋅
∂
𝐿
∂
𝑤
म
=
0.10
−
0.1
×
(
−
0.5204
)
=
0.10
+
0.05204
=
0.15204
w
म_new
	​

=w
म_old
	​

−η⋅
∂w
म
	​

∂L
	​

=0.10−0.1×(−0.5204)=0.10+0.05204=0.15204

Result: wₘ increased from 0.10 → 0.15204.
Intuition: the model underpredicted "म" (p≈0.13), so the gradient pushes wₘ up so "म" becomes more likely next time.

Quick note about other weights (like wₙ now)

At timestep t=1 the true target is "म", not "न".

The logits of other classes (including "न") still influence the denominator and thereby their gradients, but the strongest direct gradient pushes come from the true class ("म").

For a non-target class k (e.g. "न"), gradient on its logit is 
𝑝
(
𝑘
)
−
0
p(k)−0 = 
𝑝
(
𝑘
)
p(k), which typically produces a small positive gradient that tends to decrease that class’s relative logit when applied through weight updates (because the gradient will be added with a negative sign in the weight update rule for non-targets). In short: correct-class weights get boosted; many of the incorrect-class paths receive updates that reduce their relative logits.

Putting it together (two-step view)

After t=0 we updated wₙ from 0.20 → 0.2445 (from the earlier dry run).

After t=1 we updated wₘ from 0.10 → 0.1520.

Repeating this over many training examples & epochs will shape the whole weight matrix so that the decoder produces high probability for the correct next char sequence.
'''